# Code to generate folder for HMC run
May 8, 2023

In [1]:
import numpy as np
import os
import argparse
import subprocess as sp
import shutil
import glob

import yaml

In [2]:
def f_write_config(input_dict,fname):
    
    with open(fname,'w') as f:
        yaml.dump(input_dict,f,sort_keys=False)
        
def f_read_config(fname):
    
    with open(fname,'r') as f:
        config_dict=yaml.load(f, Loader=yaml.SafeLoader)
        
    return config_dict


In [3]:

def f_get_last_checkpoint(run_dir):
    '''
    Look through saved config files and find last saved configuration number
    '''
    checkpt_list=np.sort([int(i.split('/')[-1].split('_lat.')[-1]) for i in glob.glob(run_dir+'/ckpoint_lat*')])

    return checkpt_list[-1]


def f_get_out_filename(run_dir):
    '''
    Get the right file name for output such as  HSDM1.out, HSDM2.out , etc. based on number of extensions runs done
    
    '''
    count=2
    fname=run_dir+'/HSDM%s.out'%(count)

    while os.path.exists(fname): 
        count+=1
        if count>5:    
            print("Too many extend runs. Please check")
            break

        print("File exists: ",fname)
        fname=run_dir+'HSDM%s.out'%(count)
        print("Modifying output file name to ",fname)

    return fname.split('/')[-1]

In [4]:
def f_build_submit_script(dict_pars,fname,run_type):
    '''
    Build submit script for all run types
    '''
    
    
    if run_type=='fresh':
        app_strg='"$RUN_DIR/build/dweofa_mobius_HSDM --grid {Lx}.{Lx}.{Lx}.{Lt} --mpi {mpi} --shm 2048 --shm-force-mpi 1 --device-mem 5000 --StartingType {start_type} --Trajectories {total_traj} --Thermalizations {therm} $OPT $Ls $traj_l $md_steps $BETA $M_F"'.format(**dict_pars)

    elif run_type in ['checkpoint_start','extend']:
        app_strg='"$RUN_DIR/build/dweofa_mobius_HSDM --grid {Lx}.{Lx}.{Lx}.{Lt} --mpi {mpi} --shm 2048 --shm-force-mpi 1 --device-mem 5000 --StartingType {start_type} --StartingTrajectory {start_traj} --Trajectories {total_traj} --Thermalizations {therm} $OPT $Ls $traj_l $md_steps $BETA $M_F"'.format(**dict_pars)

    else: 
        print("Invalid run_type",run_type)
        raise SystemError

    submit_strg='''#!/bin/bash
################
# Begin LSF Directives
#SBATCH -A latticgc
#SBATCH -t 00:10:00
#SBATCH -J hsdm
#SBATCH -o hsdm.%J
#SBATCH -e hsdm.%J
#SBATCH -N {N}
#SBATCH -n {nprocs}
#SBATCH --exclusive
#SBATCH --gpu-bind=map_gpu:0,1,2,3,7,6,5,4
#SBATCH -c 1
###SBATCH --threads-per-core=1

echo "--start " `date` `date +%s`

export BIND="--cpu-bind=verbose,map_ldom:3,3,1,1,2,2,0,0"

export MPIR_CVAR_GPU_EAGER_DEVICE_MEM=16384
export MPICH_GPU_SUPPORT_ENABLED=1
export MPICH_SMP_SINGLE_COPY_MODE=CMA
export OMP_NUM_THREADS=8
export MPICH_OFI_NIC_POLICY=GPU
export OPT="--comms-concurrent --comms-overlap "

source $GRID_DIR/setup_env.sh
export TSAN_OPTIONS='ignore_noninstrumented_modules=1'
export LD_LIBRARY_PATH

Ls={dwf_Ls}
traj_l={traj_l}
md_steps={md_steps}
BETA={beta}
M_F={mf}

APP={app}

#srun --gpus-per-task 1 -n64 $BIND $APP > HSDM.out
#srun --gpus-per-task 1 -n64 $APP > HSDM.out
srun -n{nprocs} -o {out_file} $APP

echo "--end " `date` `date +%s`

    '''.format(app=app_strg,**dict_pars)
    with open(fname,'w') as f: f.write(submit_strg)


In [6]:
# run_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_4_cold_start/'
run_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/'
# run_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/scaling_tests/'


In [14]:
start_fldr='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.4_mf-0.1/'

assert os.path.exists(start_fldr)


In [15]:
## Define type of run

In [16]:
### Options ['fresh','extend','checkpoint_start']
    
# run_type='checkpoint_start'

run_type='extend'

# run_type='fresh'


In [17]:
## Fresh

if run_type=='fresh':

    Lx=16
    Lt=8
    N=1
    mx,my,mz,mt=2,2,2,1
    Ls=16
    dict_pars={
        'Lx':Lx, 'Lt':Lt, # Lattice size 
        'F_action': 'Mobius_dwf',

        'traj_l':2, 'md_steps':15, 
    #     'beta':beta, 'mf':mf, 
        'dwf_Ls':Ls, 
        'mpi':".".join([str(i) for i in [mx,my,mz,mt]]),
        'nprocs': mx*my*mz*mt,
        'N':N,  ## Number of nodes
        'total_traj': 200,
        'start_type': 'HotStart',   # Valid [HotStart, ColdStart, TepidStart, CheckpointStart]
        'therm': 10,
        'out_file': 'HSDM1.out'
              }

In [18]:
## checkpoint_start

if run_type=='checkpoint_start':
    
    last=f_get_last_checkpoint(start_fldr)
    print(last)
    config_file=start_fldr+'ckpoint_lat.%s'%(last)
    assert os.path.isfile(config_file) ,"File doesn't exist"
    
    Lx=24; Lt=12
    N=1
    mx,my,mz,mt=2,2,2,1
    Ls=16
    dict_pars={
        'Lx':Lx, 'Lt':Lt, # Lattice size 
        'F_action': 'Mobius_dwf',

        'traj_l':2, 'md_steps':15, 
#         'beta':14.0, 'mf':0.1, 
        'dwf_Ls':Ls, 
        'mpi':".".join([str(i) for i in [mx,my,mz,mt]]),
        'nprocs': mx*my*mz*mt,
        'N':N, 
        'total_traj': 200,
        'start_type': 'CheckpointStart',   # Valid [HotStart, ColdStart, TepidStart, CheckpointStart]
        'start_traj': 0,
        'start_config': config_file,
        'therm':0, ## Thermalization ( non-zero for fresh run )
        'out_file': 'HSDM1.out'
              }


In [19]:
## Extend run

if run_type=="extend":

    last=f_get_last_checkpoint(start_fldr)
    print(last)
    config_file=start_fldr+'ckpoint_lat.%s'%(last)
    
    print("Last checkpoint",last)

    input_dict=f_read_config(start_fldr+'/config.yaml')

    dict_pars={}
    for key in input_dict.keys():
        dict_pars[key]=input_dict[key]

    dict_pars.update({
        'start_type': 'CheckpointStart',   # Valid [HotStart, ColdStart, TepidStart, CheckpointStart]
        'start_traj': f_get_last_checkpoint(start_fldr),
        'out_file': f_get_out_filename(start_fldr),
        'total_traj': 500,
        'therm': 0,
        'starg_config': config_file,
              })

710
Last checkpoint 710
File exists:  /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.4_mf-0.1//HSDM2.out
Modifying output file name to  /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.4_mf-0.1/HSDM3.out


In [20]:
print(dict_pars)

{'Lx': 16, 'Lt': 8, 'F_action': 'Mobius_dwf', 'traj_l': 2, 'md_steps': 15, 'dwf_Ls': 16, 'mpi': '2.2.2.1', 'nprocs': 8, 'N': 1, 'start_type': 'CheckpointStart', 'beta': 11.4, 'mf': 0.1, 'start_traj': 710, 'out_file': 'HSDM3.out', 'total_traj': 500, 'therm': 0, 'starg_config': '/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.4_mf-0.1/ckpoint_lat.710'}


## Build directories and copy files

In [21]:
if run_type=='extend':
    assert os.path.exists(start_fldr), "Top directory %s doesn't exist"%(start_fldr)
    fname=start_fldr+'/submit_3_dwf-hsdm.sh'
    f_build_submit_script(dict_pars,fname,run_type)

    
elif run_type in ['fresh','checkpoint_start'] : 
    # beta_list=[10.0+0.2*i for i in range(11)]
    beta_list=[11.3]
    m_list=[0.1]

    for beta in beta_list:
        for mf in m_list:
            dict_pars['beta']=beta
            dict_pars['mf']=mf

            strg='run_Lx-%s_Lt-%s_Ls-%s_beta-%s_mf-%s'%(Lx,Lt,Ls,beta,mf)
            fldr=run_dir+'%s/'%(strg)

            print(fldr)

            if os.path.exists(fldr): 
                print("Error: Directory %s exists",fldr)
                raise SystemError
            else:
                os.makedirs(run_dir+'/{0}'.format(strg))
                fname=fldr+'submit_2_dwf-hsdm.sh'
                f_build_submit_script(dict_pars,fname,run_type)

                fname=fldr+'config.yaml'
                f_write_config(dict_pars,fname)

            if run_type=='checkpoint_start':
                print("Copy start config from the last config %s in %s"%(last,start_fldr))
                shutil.copy(start_fldr+'/'+'ckpoint_lat.%s'%(last), fldr+'/ckpoint_lat.0')
                shutil.copy(start_fldr+'/'+'ckpoint_rng.%s'%(last), fldr+'/ckpoint_rng.0')
                
                
else: 
    print("Invalid run_type",run_type)
    raise SystemError

### Code to rename folders
test_dir=run_dir+'../new_runs/test/'
os.chdir(test_dir)
fldr_lst=glob.glob('run_L*')
print(fldr_lst)
for src in fldr_lst:
     target=src.replace('run_Ls','run_Lx')
     print(src,target)
     os.rename(src,target)